In [3]:
!pip install selenium

   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.5 MB 8.3 MB/s eta 0:00:02
   ------------ --------------------------- 2.9/9.5 MB 9.9 MB/s eta 0:00:01
   -------------------- ------------------- 5.0/9.5 MB 10.8 MB/s eta 0:00:01
   --------------------------- ------------ 6.6/9.5 MB 9.8 MB/s eta 0:00:01
   ------------------------------------- -- 8.9/9.5 MB 9.5 MB/s eta 0:00:01
   ---------------------------------------- 9.5/9.5 MB 9.6 MB/s eta 0:00:00
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0


In [25]:
!pip install webdriver-manager

In [12]:
!pip install chromedriver


In [16]:
!chromedriver --version

"chromedriver" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [25]:
import time
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [102]:

# Configuración de Selenium
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Paso 1: Obtener info desde la página principal
url = "https://www.imdb.com/chart/toptv/"
driver.get(url)
time.sleep(3)

series_info = []

# Cada fila de la tabla de series
rows = driver.find_elements(By.CSS_SELECTOR, 'li.ipc-metadata-list-summary-item')[:10]

for row in rows:
    try:
        link_element = row.find_element(By.CSS_SELECTOR, "a.ipc-title-link-wrapper")
        link = link_element.get_attribute("href")
        title = link_element.find_element(By.TAG_NAME, "h3").text
    except:
        link = ""
        title = "Título no encontrado"

    try:
        rating = row.find_element(By.CSS_SELECTOR, "span.ipc-rating-star--rating").text
    except:
        rating = "Rating no disponible"

    try:
        metadata_div = row.find_element(By.CSS_SELECTOR, "div.cli-title-metadata")
        spans = metadata_div.find_elements(By.CSS_SELECTOR, "span.cli-title-metadata-item")
        años_al_aire = spans[0].text if len(spans) > 0 else "No disponible"
        episodios = spans[1].text if len(spans) > 1 else "No disponible"
    except:
        años_al_aire = "No disponible"
        episodios = "No disponible"

    series_info.append({
        "titulo": title,
        "link": link,
        "rating": rating,
        "años_al_aire": años_al_aire,
        "episodios": episodios
    })

# Función para extraer comentarios desde una URL
def obtener_comentarios(review_url, N=100):
    driver.get(review_url)
    time.sleep(5)
    all_comments = []

    try:
        todo_button = driver.find_element(By.XPATH, "//button[.//span[text()='Todo']]")
        driver.execute_script("arguments[0].click();", todo_button)
        time.sleep(5)
        print("Se hizo clic en el botón 'Todo'.")
    except:
        print("No se encontró el botón 'Todo'. Intentando con el de '25 más'...")

    # Hacer clic una vez en "1 más" si aparece
    try:
        mas_uno_button = driver.find_element(By.XPATH, "//span[text()='1 más']")
        driver.execute_script("arguments[0].click();", mas_uno_button)
        print("Se hizo clic en '1 más'.")
        time.sleep(5)
    except:
        pass

    comments = driver.find_elements(By.CSS_SELECTOR, 'div.ipc-html-content-inner-div')

    for comment in comments:
        all_comments.append(comment.text)
        if len(all_comments) >= N:
            break

    return all_comments

# Crear lista de diccionarios para convertir a DataFrame
data = []

#Recorrer los links y extraer comentarios
for serie in series_info:
    link = serie["link"]
    title = serie["titulo"]
    rating = serie["rating"]
    años_al_aire = serie["años_al_aire"]
    episodios = serie["episodios"]

    match = re.search(r'/title/(tt\d+)/', link)
    if match:
        imdb_id = match.group(1)
        review_url = f"https://www.imdb.com/title/{imdb_id}/reviews"
        print(f"\n[{title}] Visitando página de reviews: {review_url}")
        comentarios = obtener_comentarios(review_url)
        print(f"Comentarios extraídos: {len(comentarios)}")

        # Agregar al dataset
        data.append({
            "titulo": title,
            "link": review_url,
            "rating": rating,
            "años_al_aire": años_al_aire,
            "episodios": episodios,
            "comentarios": comentarios
        })
    else:
        print("No se pudo extraer el ID de la serie.")

# Crear DataFrame
df = pd.DataFrame(data)

# Mostrar las primeras filas
print(df.head())

# Guardar dataframe como csv
df.to_csv("comentarios_series.csv", index=False)


[1. Breaking Bad] Visitando página de reviews: https://www.imdb.com/title/tt0903747/reviews
Se hizo clic en el botón 'Todo'.
Comentarios extraídos: 100

[2. Planeta Tierra II] Visitando página de reviews: https://www.imdb.com/title/tt5491994/reviews
Se hizo clic en el botón 'Todo'.
Comentarios extraídos: 100

[3. Planeta Tierra] Visitando página de reviews: https://www.imdb.com/title/tt0795176/reviews
Se hizo clic en el botón 'Todo'.
Se hizo clic en '1 más'.
Comentarios extraídos: 100

[4. Banda de hermanos] Visitando página de reviews: https://www.imdb.com/title/tt0185906/reviews
Se hizo clic en el botón 'Todo'.
Comentarios extraídos: 100

[5. Chernobyl] Visitando página de reviews: https://www.imdb.com/title/tt7366338/reviews
Se hizo clic en el botón 'Todo'.
Comentarios extraídos: 100

[6. The Wire] Visitando página de reviews: https://www.imdb.com/title/tt0306414/reviews
Se hizo clic en el botón 'Todo'.
Comentarios extraídos: 100

[7. Avatar: La leyenda de Aang] Visitando página de